In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019

In [2]:
%%time
import static_individual_estimates
static_individual_estimates.estimate_paces(ve_or_ju)

2019-10-30 22:36:41,550 None
2019-10-30 22:36:41,623 None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15019 entries, 0 to 15018
Data columns (total 21 columns):
mean_team_id           15019 non-null float64
teams                  15019 non-null object
name                   15019 non-null object
num_runs               15019 non-null int64
num_valid_times        15019 non-null int64
mean_pace              14600 non-null float64
stdev                  14600 non-null float64
log_stdev              15019 non-null float64
most_common_leg        15019 non-null int64
most_common_country    15019 non-null object
pace_1                 14600 non-null float64
pace_2                 8344 non-null float64
pace_3                 5957 non-null float64
pace_4                 4225 non-null float64
pace_5                 3038 non-null float64
pace_6                 2051 non-null float64
pace_7                 1223 non-null float64
pace_8                 463 non-null float64
leg_nro                15019 non-null int64
team_id                15019 non-nul

2019-10-30 22:36:41,952 []
2019-10-30 22:36:41,963 
(15019, 41)
2019-10-30 22:36:42,383     log_q_low  predicted  log_q_high  log_std
0       1.844      6.952       2.043    0.100
1       1.858      6.929       2.076    0.109
2       1.796      7.127       2.110    0.157
3       1.806      6.927       2.046    0.120
4       1.932      7.984       2.144    0.106
5       1.819      6.723       2.039    0.110
6       1.833      7.058       2.060    0.114
7       1.826      6.728       2.016    0.095
8       1.852      6.860       2.009    0.078
9       1.863      7.041       2.102    0.119
10      1.820      6.810       2.054    0.117
11      1.866      6.923       2.029    0.081
12      1.831      6.726       2.035    0.102
13      1.890      7.582       2.169    0.140
14      1.830      7.194       2.099    0.134
2019-10-30 22:36:42,386 0.17930748029502808
2019-10-30 22:36:42,392 prior_mean_error mean: 1.4465
2019-10-30 22:36:42,394 prior_mean_error_in_sd mean: 1.1942
/Users/oskari/.vir

CPU times: user 1.02 s, sys: 71.8 ms, total: 1.09 s
Wall time: 1.1 s


In [3]:
%%time
static_individual_estimates.combine_estimates_with_running_order(ve_or_ju)

2019-10-30 22:36:42,606 
(6759, 7)
2019-10-30 22:36:42,636 
(15019, 12)


Found 1 history with switched name evely kaasiku 
Found 1 history with switched name johanna hulkkonen 
Found 1 history with switched name miia niittynen 
Found 3 history with switched name svetlana mironova 
Found 1 history with switched name karoliina ukskoski 
Found 1 history with switched name marjo taanila 
Found 1 history with switched name ulrika uotila 
Found 1 history with switched name zhanalova barbora 
Found 1 history with switched name nymark tora 
Found 1 history with switched name boman monica 
Found 1 history with switched name oikarinen anni 
name 'sanna timonen' team_name 'MAASOTAKOULU LPR'
by_name 0 by_name_and_colon 5 runners 2
Duplicate runner       Unnamed: 0  mean_team_id  num_valid_times  mean_pace  stdev  log_stdev  \
7156        7156         501.0                1     13.403    0.0        0.0   
7159        7159         777.0                1     10.631    0.0        0.0   

      prior_mean  prior_log_std  predicted_log_pace_mean  \
7156     10.9190         0

2019-10-30 22:37:37,567 None
2019-10-30 22:37:37,599 None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6759 entries, 0 to 6758
Data columns (total 12 columns):
team_id           6759 non-null int64
team              6759 non-null object
team_base_name    6759 non-null object
team_country      6759 non-null object
leg               6759 non-null int64
leg_dist          6759 non-null float64
name              6759 non-null object
leg_nro           6759 non-null int64
orig_name         6759 non-null object
num_runs          6759 non-null float64
pred_log_mean     6725 non-null float64
pred_log_std      6725 non-null float64
dtypes: float64(4), int64(3), object(5)
memory usage: 633.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 4 columns):
first_name           974 non-null object
mean_pace_count      975 non-null int64
fn_pace_class        975 non-null int64
fn_pace_std_class    975 non-null int64
dtypes: int64(3), object(1)
memory usage: 30.6+ KB


2019-10-30 22:37:37,771 []
2019-10-30 22:37:37,840 
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6759 entries, 0 to 6758
Data columns (total 41 columns):
team_id                  -6759 non-null int64
team_id_log10            -6759 non-null float64
team_id_square           -6759 non-null int64
leg_1                    -6759 non-null Sparse[uint8, 0]
leg_2                    -6759 non-null Sparse[uint8, 0]
leg_3                    -6759 non-null Sparse[uint8, 0]
leg_4                    -6759 non-null Sparse[uint8, 0]
c_EST                    -6759 non-null Sparse[uint8, 0]
c_FIN                    -6759 non-null Sparse[uint8, 0]
c_GBR                    -6759 non-null Sparse[uint8, 0]
c_LAT                    -6759 non-null Sparse[uint8, 0]
c_NOR                    -6759 non-null Sparse[uint8, 0]
c_OTHER                  -6759 non-null Sparse[uint8, 0]
c_RUS                    -6759 non-null Sparse[uint8, 0]
c_SUI                    -6759 non-null Sparse[uint8, 0]
c_SWE                    -6759 non-null Sparse[uint8, 0]
runs_1.0    

2019-10-30 22:37:38,070     log_q_low  predicted  log_q_high  log_std
0       1.861      7.209       2.084    0.111
1       1.814      6.642       2.010    0.098
2       1.796      6.990       2.105    0.154
3       1.799      6.618       1.991    0.096
4       1.860      6.939       2.032    0.086
5       1.807      6.873       2.027    0.110
6       1.785      6.877       2.073    0.144
7       1.840      7.339       2.013    0.086
8       1.856      6.892       2.016    0.080
9       1.813      6.608       2.004    0.096
10      1.836      7.017       2.119    0.142
11      1.843      6.879       1.968    0.062
12      1.845      7.141       2.052    0.103
13      1.835      7.179       2.093    0.129
14      1.801      7.031       2.072    0.136
2019-10-30 22:37:38,074 0.17501892910330646
2019-10-30 22:37:38,086 
count    6759.000000
mean        0.175019
std         0.034479
min         0.006586
1%          0.089751
5%          0.116020
25%         0.151953
50%         0.175901
75%

CPU times: user 55.3 s, sys: 329 ms, total: 55.7 s
Wall time: 55.7 s


In [4]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
display(running_order.shape)



(6759, 20)

In [5]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std", "num_runs"]]


In [6]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std,num_runs
0,1,Stora Tuna OK 1,1,6.0,Tilda Östberg,1.8857,0.111400,1.0
1,1,Stora Tuna OK 1,2,5.7,Magdalena Olsson,1.9587,0.082400,7.0
2,1,Stora Tuna OK 1,3,7.3,Julia Jakob,1.7824,0.154366,1.0
3,1,Stora Tuna OK 1,4,7.9,Tove Alexandersson,1.7963,0.067659,7.0
4,2,Göteborg Majorna OK 1,1,6.0,Amanda Hansson,2.0053,0.086461,2.0


In [7]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                                \
leg                          1                      2                      3   
team_id                                                                        
1              Stora Tuna OK 1        Stora Tuna OK 1        Stora Tuna OK 1   
2        Göteborg Majorna OK 1  Göteborg Majorna OK 1  Göteborg Majorna OK 1   
3                   MS Parma 1             MS Parma 1             MS Parma 1   
4          Järla Orientering 1    Järla Orientering 1    Järla Orientering 1   
5               Alfta ÖSA OK 1         Alfta ÖSA OK 1         Alfta ÖSA OK 1   

                               dist                               name  \
leg                          4    1    2    3    4                   1   
team_id                                                                  
1              Stora Tuna OK 1  6.0  5.7  7.3  7.9       Tilda Östberg   
2        Göteborg Majorna OK 1  6.0  5.7  7.3  7.9      Amanda Hansson   
3                   MS Parma 1  6.0  5.7  7.3  7.9   Anastasia Rudnaya   
4          Järla Orientering 1  6.0  5.7  7.3  7.9       Sara Sjökvist   
5               Alfta ÖSA OK 1  6.0  5.7  7.3  7.9  Galina Vinogradova   

                              ... log_mean           log_std            \
leg                        2  ...        3       4         1         2   
team_id                       ...                                        
1           Magdalena Olsson  ...   1.7824  1.7963  0.111400  0.082400   
2              Runa Fremstad  ...   1.9581  1.9854  0.086461  0.087900   
3             Anni Heikkonen  ...   1.8669  1.9113  0.067659  0.100600   
4        Anna Dahlgren-Rosén  ...   1.8086  1.9289  0.112800  0.182322   
5            Josefine Heikka  ...   1.9000  1.8720  0.102900  0.094482   

                            num_runs                 
leg             3         4        1    2    3    4  
team_id                                              
1        0.154366  0.067659      1.0  7.0  1.0  7.0  
2        0.067659  0.086115      2.0  5.0  5.0  3.0  
3        0.141629  0.067659      4.0  7.0  1.0  8.0  
4        0.135658  0.108649      4.0  0.0  1.0  2.0  
5        0.095300  0.128324      7.0  2.0  6.0  3.0  

[5 rows x 24 columns]

In [8]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(1697, 24)

(1682, 24)

In [9]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

270.0

In [10]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [11]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    personal_start95 = samples.quantile(0.025)
    personal_end95 = samples.quantile(0.975)
    personal_median = samples.median()
        
    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    personal_start95_dict = {}
    personal_end95_dict = {}
    personal_median_dict = {}
    for leg in range(1, num_legs + 1):
        personal_start95_dict[f"personal_start95_{leg}"] = personal_start95.values[leg-1]
        personal_end95_dict[f"personal_end95_{leg}"] = personal_end95.values[leg-1]
        personal_median_dict[f"personal_median_{leg}"] = personal_median.values[leg-1]
        
    new_cols = {**fin_start95_dict, 
                **fin_end95_dict, 
                **fin_median_dict, 
                **fin_mean_dict, 
                **fin_sum_log_means_dict, 
                **fin_sum_log_stds_dict, 
                **mass_start_dict, 
                **dark_during_leg_dict,
                **personal_start95_dict,
                **personal_end95_dict,
                **personal_median_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,31.712798,70.015926,108.043883,154.558894,49.024594,91.754692,142.367420,191.153292,39.473979,80.067541,...,32.056818,41.769169,49.024594,47.423441,58.747664,54.451147,39.473979,40.405797,43.315946,47.600038
2,37.667435,77.270922,127.298471,181.800271,52.761805,98.405993,152.016669,213.412380,44.568890,87.232520,...,45.300509,48.785576,52.761805,50.361495,59.136839,68.002100,44.568890,42.438528,51.780661,57.500309
3,34.725531,70.828085,112.099142,164.153965,45.248329,89.596019,144.905366,199.805621,39.685716,79.582616,...,35.626289,46.869007,45.248329,48.286271,62.713381,61.009992,39.685716,39.798988,47.204038,53.352030
4,34.668221,68.937125,109.953429,161.439848,53.804441,104.554667,153.296696,210.709171,43.228280,84.430796,...,34.209381,43.844848,53.804441,58.532021,58.252847,67.582700,43.228280,40.783966,44.555718,54.430840
5,32.461618,67.400765,113.199805,160.504617,48.674232,88.907543,141.739160,198.799560,39.665595,77.578896,...,40.393739,40.065224,48.674232,45.102533,59.063945,66.216125,39.665595,37.631358,48.789559,51.436902


In [12]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat.head().round(2)

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4,num_runs_1,num_runs_2,num_runs_3,num_runs_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,1.78,1.80,0.11,0.08,0.15,0.07,1.0,7.0,1.0,7.0
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,1.96,1.99,0.09,0.09,0.07,0.09,2.0,5.0,5.0,3.0
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,1.87,1.91,0.07,0.10,0.14,0.07,4.0,7.0,1.0,8.0
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,1.81,1.93,0.11,0.18,0.14,0.11,4.0,0.0,1.0,2.0
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,1.90,1.87,0.10,0.09,0.10,0.13,7.0,2.0,6.0,3.0


In [13]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [14]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,32.056818,41.769169,49.024594,47.423441,58.747664,54.451147,39.473979,40.405797,43.315946,47.600038
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,45.300509,48.785576,52.761805,50.361495,59.136839,68.002100,44.568890,42.438528,51.780661,57.500309
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,35.626289,46.869007,45.248329,48.286271,62.713381,61.009992,39.685716,39.798988,47.204038,53.352030
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,34.209381,43.844848,53.804441,58.532021,58.252847,67.582700,43.228280,40.783966,44.555718,54.430840
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,40.393739,40.065224,48.674232,45.102533,59.063945,66.216125,39.665595,37.631358,48.789559,51.436902


In [15]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9587   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.7963   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   
        2    Göteborg Majorna OK 1   5.7       Runa Fremstad    2.0081   
        3    Göteborg Majorna OK 1   7.3    Kristine Berglia    1.9581   

              log_std  num_runs  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
1       1    0.111400       1.0    31.712798   49.024594   39.473979   
        2    0.082400       7.0    70.015926   91.754692   80.067541   
        3    0.154366       1.0   108.043883  142.367420  123.706352   
        4    0.067659       7.0   154.558894  191.153292  171.539565   
2       1    0.086461       2.0    37.667435   52.761805   44.568890   
        2    0.087900       5.0    77.270922   98.405993   87.232520   
        3    0.067659       5.0   127.298471  152.016669  139.191360   

               fin_mean  fin_sum_log_mean  fin_sum_log_std  mass_start  \
team_id leg                                                              
1       1     39.772385          3.676981         0.111345         0.0   
        2     80.311791          4.383546         0.068827         0.0   
        3    124.157172          4.819059         0.070516         0.0   
        4    171.899777          5.145447         0.054086         0.0   
2       1     44.753690          3.797442         0.086353         0.0   
        2     87.400934          4.468595         0.061834         0.0   
        3    139.279482          4.935428         0.045933         0.0   

             dark_during_leg  personal_start95  personal_end95  \
team_id leg                                                      
1       1                0.0         31.712798       49.024594   
        2                0.0         34.404788       47.423441   
        3                0.0         32.056818       58.747664   
        4                0.0         41.769169       54.451147   
2       1                0.0         37.667435       52.761805   
        2                0.0         35.773646       50.361495   
        3                0.0         45.300509       59.136839   

             personal_median  
team_id leg                   
1       1          39.473979  
        2          40.405797  
        3          43.315946  
        4          47.600038  
2       1          44.568890  
        2          42.438528  
        3          51.780661

In [16]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

,,name,fin_start95,fin_median,fin_end95,mass_start,dark_during_leg
team_id,leg,,,,,,


In [17]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]
local_start_ts = pd.Timestamp(start_timestamp).tz_localize(None)

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= local_start_ts)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= local_start_ts)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= local_start_ts)

runner_estimates["personal_estimate_interval"] = runner_estimates["personal_end95"] - runner_estimates["personal_start95"]
runner_estimates.head()

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9587   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.7963   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   

              log_std  num_runs  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
1       1    0.111400       1.0    31.712798   49.024594   39.473979   
        2    0.082400       7.0    70.015926   91.754692   80.067541   
        3    0.154366       1.0   108.043883  142.367420  123.706352   
        4    0.067659       7.0   154.558894  191.153292  171.539565   
2       1    0.086461       2.0    37.667435   52.761805   44.568890   

               fin_mean  ...  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg              ...                                                 
1       1     39.772385  ...         0.111345         0.0              0.0   
        2     80.311791  ...         0.068827         0.0              0.0   
        3    124.157172  ...         0.070516         0.0              0.0   
        4    171.899777  ...         0.054086         0.0              0.0   
2       1     44.753690  ...         0.086353         0.0              0.0   

             personal_start95  personal_end95  personal_median  \
team_id leg                                                      
1       1           31.712798       49.024594        39.473979   
        2           34.404788       47.423441        40.405797   
        3           32.056818       58.747664        43.315946   
        4           41.769169       54.451147        47.600038   
2       1           37.667435       52.761805        44.568890   

                          fin_time_median              fin_time_start95  \
team_id leg                                                               
1       1   2019-06-15 14:39:28.438755274 2019-06-15 14:31:42.767865419   
        2   2019-06-15 15:20:04.052478313 2019-06-15 15:10:00.955543756   
        3   2019-06-15 16:03:42.381123304 2019-06-15 15:48:02.633005142   
        4   2019-06-15 16:51:32.373878717 2019-06-15 16:34:33.533654928   
2       1   2019-06-15 14:44:34.133405924 2019-06-15 14:37:40.046104908   

                           fin_time_end95 personal_estimate_interval  
team_id leg                                                           
1       1   2019-06-15 14:49:01.475619078                  17.311796  
        2   2019-06-15 15:31:45.281549692                  13.018653  
        3   2019-06-15 16:22:22.045193434                  26.690846  
        4   2019-06-15 17:11:09.197546482                  12.681978  
2       1   2019-06-15 14:52:45.708304405                  15.094370  

[5 rows x 21 columns]

In [18]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")

for_print[for_print['team'].str.contains("Stadin")]
#for_print = for_print[for_print['team'].str.contains("Puskasilimät")]

for_print = for_print[[
 'leg',
 'name', 
 'num_runs', 
 'personal_start95', 
 'personal_median',
 'personal_end95',
 'personal_estimate_interval']]

for_print.to_csv(f'data/for_print_{ve_or_ju}_2019.csv')
for_print

,leg,name,num_runs,personal_start95,personal_median,personal_end95,personal_estimate_interval
team_id,,,,,,,
1,1,Tilda Östberg,1.0,31.71,39.47,49.02,17.31
1,2,Magdalena Olsson,7.0,34.40,40.41,47.42,13.02
1,3,Julia Jakob,1.0,32.06,43.32,58.75,26.69
1,4,Tove Alexandersson,7.0,41.77,47.60,54.45,12.68
2,1,Amanda Hansson,2.0,37.67,44.57,52.76,15.09
...,...,...,...,...,...,...,...
1722,4,Johanna Liinamaa,3.0,72.53,89.33,109.92,37.39
1723,1,Elina Holstikko,0.0,63.17,90.31,128.98,65.80
1723,2,Mirka Leinonen,0.0,59.83,85.77,122.93,63.10


In [19]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'num_runs',     
 'ind_log_mean', 
 'ind_log_std',
 'personal_start95', 
 'personal_median',
 'personal_end95',    
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Downt")]

,team_id,leg,team,name,num_runs,ind_log_mean,ind_log_std,personal_start95,personal_median,personal_end95,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg
1332,341,1,Rasti-Jyry 1,Jola Maisala,8.0,2.188600,0.126500,41.683227,53.478017,68.772243,3.981103,0.127204,2019-06-15 11:41:40.993634462+00:00,2019-06-15 12:08:46.334605932+00:00,2019-06-15 11:53:28.681005239+00:00,0.0000,0.0
1333,341,2,Rasti-Jyry 1,Kaija Rantoja,7.0,2.296600,0.144100,42.565712,56.670530,75.175978,4.706924,0.097184,2019-06-15 12:31:23.507049084+00:00,2019-06-15 13:13:39.223928690+00:00,2019-06-15 12:50:53.305011272+00:00,0.0000,0.0
1334,341,3,Rasti-Jyry 1,Jenni Castren,5.0,2.334200,0.141700,57.212989,75.479386,99.833800,5.229857,0.081366,2019-06-15 13:39:36.437912941+00:00,2019-06-15 14:39:20.439147711+00:00,2019-06-15 14:06:39.967589855+00:00,0.0000,0.0
1335,341,4,Rasti-Jyry 1,Inka Lemmetyinen,1.0,2.293200,0.168000,56.083801,78.368310,107.892492,5.583406,0.076261,2019-06-15 14:49:29.424722672+00:00,2019-06-15 16:08:35.336018801+00:00,2019-06-15 15:26:07.676917553+00:00,0.0000,0.0
2076,528,1,Rasti-Jyry 2,Päivi Taponen,3.0,2.361600,0.167728,45.536115,63.518151,88.431981,4.151770,0.168782,2019-06-15 11:45:32.166923523+00:00,2019-06-15 12:28:25.918869019+00:00,2019-06-15 12:03:31.089051962+00:00,0.0000,0.0
2077,528,2,Rasti-Jyry 2,Mervi Luurila,2.0,2.473200,0.183049,47.153141,67.487877,97.291232,4.882773,0.125149,2019-06-15 12:43:10.508324385+00:00,2019-06-15 13:49:11.274507761+00:00,2019-06-15 13:11:56.257523537+00:00,0.0000,0.0
2078,528,3,Rasti-Jyry 2,Suvi Tiainen,2.0,2.646600,0.187243,71.871388,103.350037,149.953272,5.467708,0.107972,2019-06-15 14:12:50.835812807+00:00,2019-06-15 15:53:59.039696932+00:00,2019-06-15 14:56:26.923872232+00:00,0.0000,0.0
2079,528,4,Rasti-Jyry 2,Tiina Isoniemi,6.0,2.153800,0.067659,59.717249,68.009667,77.522915,5.721630,0.085060,2019-06-15 15:19:38.608668327+00:00,2019-06-15 17:02:14.651145220+00:00,2019-06-15 16:04:58.432914972+00:00,0.1142,0.0
2916,740,1,Rasti-Jyry 4,Mila Viksilä,0.0,2.287041,0.182322,41.650947,59.469706,84.788115,4.084184,0.183311,2019-06-15 11:41:39.056828499+00:00,2019-06-15 12:24:47.286902428+00:00,2019-06-15 11:59:28.182353020+00:00,0.0000,0.0
2917,740,2,Rasti-Jyry 4,Tiina Sidensnöre,7.0,2.262600,0.146600,40.969438,54.651152,72.741243,4.743813,0.118467,2019-06-15 12:30:49.024215698+00:00,2019-06-15 13:24:32.844667196+00:00,2019-06-15 12:54:53.549423933+00:00,0.0000,0.0


In [20]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")

In [21]:
for_web.shape

(6728, 17)

In [22]:
for_print[['num_runs', 'personal_median', "personal_estimate_interval"]].groupby('num_runs').agg(["mean", "count"]).round(2)


personal_median       personal_estimate_interval      
                    mean count                       mean count
num_runs                                                       
0.0                82.51  1646                      65.63  1646
1.0                80.73  1021                      61.90  1021
2.0                81.43   730                      58.05   730
3.0                78.10   697                      55.98   697
4.0                76.82   557                      32.73   557
5.0                75.57   566                      35.21   566
6.0                73.00   551                      33.59   551
7.0                71.34   571                      32.77   571
8.0                71.38   386                      32.30   386
9.0                84.02     2                      36.82     2
10.0               62.41     1                      16.70     1

In [23]:
for_web["interval_length"] = for_web.fin_time_end95 - for_web.fin_time_start95
for_web["interval_length"].median()
#for_web[['leg', 'interval_length', "num_runs"]].info()

Timedelta('0 days 01:08:57.222907')

In [24]:
for_web[['leg', 'interval_length', "num_runs"]].groupby('leg').agg(["mean", "count"]).round(2)

num_runs      
        mean count
leg               
1       2.97  1682
2       2.75  1682
3       2.98  1682
4       3.23  1682

In [25]:
shared.log_df("DONE")

2019-10-30 22:39:04,568 DONE
